In [6]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4, A3
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.plot import show
from PIL import Image
from matplotlib_scalebar.scalebar import ScaleBar
import geopandas as gpd
from pathlib import Path
from datetime import datetime
import re
import json

In [3]:
# Get images directory and centre point layer for plot title info
with open("config.json", "r") as file:
    config = json.load(file)

output_folder = Path(config["output_dir"])

input_file_gdb = Path(config["input_gdb"])

sites_gdf = gpd.read_file(input_file_gdb, layer="Registered_Sites_Merged_v2")

## Current Process - One PDF per image id with 1 - 2 images per site

In [4]:
def extract_datetime(file_path):
    # Assuming the datetime format is consistent and always at the same position in the file name
    try:
        datetime_str = file_path.stem.split("_")[3]
        # Assuming the format is YYYYMMDDHHMMSSFFF (Year, Month, Day, Hour, Minute, Second, Fraction of second)
        return datetime.strptime(datetime_str, "%Y%m%d%H%M%S%f")
    except:
        return None


def pdf_per_image_id(images_folder, pdf_dir, gdf):
    """Create one pdf for all tifs of image id

    Image id is the id part at the end of the file name <file name>_<id>.tif
    """

    images = list(Path(images_folder).glob("*.tif"))
    image_ids = list(set([int(img.stem.split("_")[-1]) for img in images]))

    for image_id in image_ids:
        selected_images = [
            img for img in images if int(img.stem.split("_")[-1]) == image_id
        ]
        selected_images.sort(key=extract_datetime, reverse=True)

        pdf_path = pdf_dir / f"pleiades_quarry_id_{image_id}.pdf"
        c = canvas.Canvas(str(pdf_path), pagesize=A3)
        page_width, page_height = A3

        for selected_image in selected_images:
            img_datetime = extract_datetime(selected_image)
            if img_datetime:
                formatted_date = img_datetime.strftime("%d %b %Y")
            else:
                formatted_date = "Unknown Date"
            # Extract image_id from the file name (assuming format "_<id>.tif")
            image_id = selected_image.stem.split("_")[-1]

            # Filter the GeoDataFrame for the current image_id
            gdf_filtered = gdf[gdf["image_id"] == int(image_id)]
            if not gdf_filtered.empty:
                business_name = gdf_filtered["USER_Name_of_business"].iloc[0]
                title = f"{image_id} - {business_name} ({formatted_date})"
            else:
                title = f"{image_id} ({formatted_date})"

            with rio.open(selected_image) as src:

                fig, ax = plt.subplots(figsize=(18, 18))

                ax.set_title(title, pad=20)
                show(
                    src, ax=ax, with_bounds=False
                )  # with_bounds=False to not alter aspect ratio

                # Add a scale bar
                scalebar = ScaleBar(0.5, units="m", location="lower right")
                ax.add_artist(scalebar)

                ax.set_axis_off()

                temp_png_path = selected_image.with_suffix(".png")
                plt.savefig(temp_png_path, dpi=100)  # Specify DPI for image quality
                plt.close(fig)

                with Image.open(temp_png_path) as img:
                    img_width, img_height = img.size

                # Calculate the image size to fit in the PDF while preserving aspect ratio
                margin = 10
                max_width = page_width - (2 * margin)
                max_height = page_height - (2 * margin)

                aspect_ratio = img_width / img_height
                if (max_width / max_height) > aspect_ratio:
                    img_width = aspect_ratio * max_height
                    img_height = max_height
                else:
                    img_width = max_width
                    img_height = img_width / aspect_ratio

                x_position = (page_width - img_width) / 2
                y_position = (page_height - img_height) / 2

                # Draw the image centered on the page
                c.drawImage(
                    str(temp_png_path),
                    x_position,
                    y_position,
                    width=img_width,
                    height=img_height,
                    preserveAspectRatio=True,
                )
                c.showPage()

                temp_png_path.unlink()

        c.save()

In [7]:
# Directory of images
images_folder = output_folder / "extracted_images"

# Output PDF path
pdf_dir = output_folder

# Create PDFs
pdf_per_image_id(images_folder, pdf_dir, sites_gdf)

## Previous process - one PDF for all images

In [6]:
def add_images_to_pdf(images_folder, pdf_path, gdf):
    """Plot all images in a directory to PDF"""
    c = canvas.Canvas(str(pdf_path), pagesize=A4)
    page_width, page_height = A4

    images = list(Path(images_folder).glob("*.tif"))
    images.sort(key=lambda img: int(img.stem.split("_")[-1]))

    for image_path in images:
        match = re.search(r"\d{14}", image_path.stem)
        if match:
            # Extract the first 8 digits (YYYYMMDD) from the match for the date
            datetime_str = match.group(0)[:8]
            datetime_obj = datetime.strptime(datetime_str, "%Y%m%d")
            formatted_date = datetime_obj.strftime("%d %b %Y")
        else:
            formatted_date = "Unknown Date"
        # Extract image_id from the file name (assuming format "_<id>.tif")
        image_id = image_path.stem.split("_")[-1]

        # Filter the GeoDataFrame for the current image_id
        gdf_filtered = gdf[gdf["image_id"] == int(image_id)]
        if not gdf_filtered.empty:
            business_name = gdf_filtered["USER_Name_of_business"].iloc[0]
            title = f"{image_id} - {business_name} ({formatted_date})"
        else:
            title = f"{image_id} ({formatted_date})"

        with rio.open(image_path) as src:

            fig, ax = plt.subplots(figsize=(9, 9))

            ax.set_title(title, pad=20)
            show(
                src, ax=ax, with_bounds=False
            )  # with_bounds=False to not alter aspect ratio

            # Add a scale bar
            scalebar = ScaleBar(0.5, units="m", location="lower right")
            ax.add_artist(scalebar)

            ax.set_axis_off()

            temp_png_path = image_path.with_suffix(".png")
            plt.savefig(temp_png_path, dpi=100)  # Specify DPI for image quality
            plt.close(fig)

            image_width_in_points = 9 * 72
            image_height_in_points = 9 * 72
            x_position = (page_width - image_width_in_points) / 2
            y_position = (page_height - image_height_in_points) / 2

            # Draw the image centered on the page
            c.drawImage(
                str(temp_png_path),
                x_position,
                y_position,
                width=image_width_in_points,
                height=image_height_in_points,
                preserveAspectRatio=True,
            )
            c.showPage()

            temp_png_path.unlink()

    c.save()

In [8]:
# Directory of images
image_folder = output_folder / "extracted_images"

# Output PDF path
pdf_path = output_folder / "output_images.pdf"

# Create PDF
add_images_to_pdf(image_folder, pdf_path, sites_gdf)